# 最简版：按工单号展示审核通过/不通过图片

只需设置 `GROUP_DATA_BASE` 和 `GROUP_ID`。


In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import gridspec
from PIL import Image as PILImage
from collections import defaultdict

# ======= 配置 =======
GROUP_DATA_BASE = Path("/data/Qwen3-VL/group_data")
GROUP_ID = "QC-20231030-0022651"
MAX_PER_CLASS = 64
COLS = 3  # 每行显示多少张

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".webp"}


def _is_image(p: Path) -> bool:
    return p.is_file() and p.suffix.lower() in IMG_EXTS


def _collect_images(d: Path):
    if not d.exists() or not d.is_dir():
        return []
    return sorted([p for p in d.iterdir() if _is_image(p)])


def _show_grid(images, title, cols=4, title_color="black"):
    if not images:
        return
    n = len(images)
    cols = max(1, min(cols, n))
    rows = (n + cols - 1) // cols

    fig = plt.figure(figsize=(cols * 3.4, rows * 3.4))
    gs = gridspec.GridSpec(rows, cols, figure=fig, hspace=0.3, wspace=0.3)

    for idx, p in enumerate(images):
        ax = fig.add_subplot(gs[idx // cols, idx % cols])
        try:
            img = PILImage.open(p)
            ax.imshow(img)
            ax.set_title(p.name, fontsize=9, fontweight="bold")
            ax.axis("off")
        except Exception as e:
            ax.text(
                0.5,
                0.5,
                f"加载失败\n{p.name}\n{e}",
                ha="center",
                va="center",
                fontsize=8,
            )
            ax.axis("off")

    fig.suptitle(
        f"{title} ({n})", fontsize=14, fontweight="bold", y=0.98, color=title_color
    )
    fig.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


def _get_mission(d: Path) -> str:
    parts = d.parts
    if "scene_2.0_order" in parts:
        idx = parts.index("scene_2.0_order")
        if idx + 1 < len(parts):
            return parts[idx + 1]
    if "rru_scene" in parts:
        return "rru_scene"
    return "unknown"


pass_by_mission = defaultdict(list)
fail_by_mission = defaultdict(list)
for d in GROUP_DATA_BASE.rglob(GROUP_ID):
    if not d.is_dir():
        continue
    parts = set(d.parts)
    mission = _get_mission(d)
    if "审核通过" in parts:
        pass_by_mission[mission].extend(_collect_images(d))
    elif "审核不通过" in parts:
        fail_by_mission[mission].extend(_collect_images(d))

if MAX_PER_CLASS and MAX_PER_CLASS > 0:
    for k in list(pass_by_mission.keys()):
        pass_by_mission[k] = pass_by_mission[k][:MAX_PER_CLASS]
    for k in list(fail_by_mission.keys()):
        fail_by_mission[k] = fail_by_mission[k][:MAX_PER_CLASS]

has_any = False
for mission in sorted(pass_by_mission.keys()):
    imgs = pass_by_mission[mission]
    if imgs:
        has_any = True
        _show_grid(imgs, f"审核通过 - {mission}", cols=COLS, title_color="green")

for mission in sorted(fail_by_mission.keys()):
    imgs = fail_by_mission[mission]
    if imgs:
        has_any = True
        _show_grid(imgs, f"审核不通过 - {mission}", cols=COLS, title_color="red")

if not has_any:
    print("未找到包含审核通过/审核不通过的图片目录，请检查 GROUP_ID 或目录结构。")

/tmp/ipykernel_207414/539583332.py:45: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0, 1, 0.96])
